In [12]:
!pip install stable-baselines3

^C


In [9]:
import numpy as np
from gym import Env
from gym.spaces import Box, Discrete 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import os
import tensorflow
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker
from stable_baselines3 import DQN


ModuleNotFoundError: No module named 'stable_baselines3'

In [3]:
drive = webdriver.Chrome(executable_path= ChromeDriverManager().install())

C:\Users\h1n1b\AppData\Local\Temp/ipykernel_19704/4283913967.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  drive = webdriver.Chrome(executable_path= ChromeDriverManager().install())


In [4]:
class webGame(Env):
    current_num = 0
    def __init__(self, drive):
        super().__init__()
        global current_num
        global current_card
        global current_player
        global current_enermy 
        self.drive = drive
        self.weapons_path = r'D:\使用者\桌面\god field\data\ weapons.npy'
        self.armor_path = r'D:\使用者\桌面\god field\data\ armor.npy'
        self.mircles_path = r'D:\使用者\桌面\god field\data\ miracles.npy'
        self.sundries_path = r'D:\使用者\桌面\god field\data\ sundries.npy'
        self.trade_path = r'D:\使用者\桌面\god field\data\ trade.npy'
        #            [+atk/-def, price, element,+hp/-mp, costMP, php, pmp, pmy, ehp, emp, emy, emy_atk  ]
        observation_high = [-60, 0, 0, -50, -50, -50, -50, -50, -50, -50, -50, 0]
        observation_low =  [ 50,50, 8,  50,  50,  50,  50, 100, 100, 100, 100, 50]
        self.observation_space = Box(np.array(observation_high), np.array(observation_low), dtype= np.int32)
        self.action_space = Discrete(3) 
        self.state = " "
        self.done = False
        self.maxnum = 0
        self.player = "CybeRacoonAI"
        self.weapons = np.load(self.weapons_path)
        self.armor = np.load(self.armor_path)
        self.mircles = np.load(self.mircles_path)
        self.sundries = np.load(self.sundries_path)
        self.trade = np.load(self.trade_path)
        self.cards = list(self.weapons)+list(self.armor)+list(self.sundries)+list(self.mircles)+list(self.trade)

    def step(self, action):
        global current_num
        global current_card
        global current_player
        global current_enermy
        # choose card
        info = {}
        reward = 0
        done_ = self.get_done()
        print("Done:", done_)
        if done_ == True:
            new_observation = [0,0,8,0,0,0,0,0,0,0,0,0]
            return new_observation, reward, done_, info
        elif done_ == None:
            sleep(2)
            if action == 0 :
                print("Action:Choose")
                card_num = self.choose_card(current_card)
                self.click_p_btn(card_num)
                reward += 1
            # pass  
            if action == 1 :
                print("Action:Next")
                current_num += 1
            # play card
            if action == 2 :
                print("Action:Play card")
                self.play_card()

            sleep(2)
            current_num += 1
            new_observation = self.observation()
        # elif new_observation[2] != 8:
            player, enermy = self.get_condition()
            player_condi = map(int, list(player.values()))
            enermy_condi = map(int, list(enermy.values()))
            self.get_state()
            if sum(player_condi) > sum(current_player):
                reward = reward+5
            else:
                reward = reward-5
            if sum(enermy_condi) < sum(current_enermy):
                reward = reward+5
            else:
                reward = reward-5
            if sum(player_condi) == 0 :
                reward -= 100
            if sum(enermy_condi) == 0 :
                reward += 100
            if self.maxnum<=current_num:
                self.maxnum = current_num
            if self.maxnum - current_num > 0:
                reward+=1
            return new_observation, reward, done_, info
        # else:
        #     new_observation = [0,0,0,0,0,0,0,0,0,0,0,0]
        #     reward = 0
        #     final_done = True
        #     return new_observation, reward, final_done, info
    def get_condition(self):
        result = ""
        e_result = ''
        for num in range(1,11):
            try:
                xpath = '//*[@id="main"]/div/div[3]/div[3]/div[6]/div['+str(num)+']/div/div/div/span[1]'
                name = self.drive.find_element('xpath',xpath )
                res = name.get_attribute('textContent')
                if res == self.player:
                    t_num = num
                else:
                    e_num = num
            except:
                pass
        for value in range(1,8):
            try:    
                v_xpath = '//*[@id="main"]/div/div[3]/div[3]/div[6]/div['+str(t_num)+']/div/div/div/span['+str(value)+']'
                # v_xpath = '//*[@id="main"]/div/div[3]/div[3]/div[6]/div[2]/div/div/div/span[1]'
                v_name = self.drive.find_element('xpath', v_xpath)
                v_res =  v_name.get_attribute('textContent')

                e_xpath = '//*[@id="main"]/div/div[3]/div[3]/div[6]/div['+str(e_num)+']/div/div/div/span['+str(value)+']'
                e_name = self.drive.find_element('xpath', e_xpath)
                e_res =  e_name.get_attribute('textContent')

                # e_result = str(e_result)+" "+e_res
                # result = str(result)+" "+v_res
                if value == 3:
                    e_HP = e_res
                    v_HP = v_res
                if value == 5:
                    e_MP = e_res
                    v_MP = v_res
                if value == 7:
                    e_MY = e_res
                    v_MY = v_res
            except UnboundLocalError:
                v_xpath = '//*[@id="main"]/div/div[3]/div[3]/div[6]/div['+str(t_num)+']/div/div/div/span['+str(value)+']'
                # v_xpath = '//*[@id="main"]/div/div[3]/div[3]/div[6]/div[2]/div/div/div/span[1]'
                v_name = self.drive.find_element('xpath', v_xpath)
                v_res =  v_name.get_attribute('textContent')
                if value == 3:
                    e_HP = 100
                    v_HP = v_res
                if value == 5:
                    e_MP = 100
                    v_MP = v_res
                if value == 7:
                    e_MY = 100
                    v_MY = v_res
            except:
                pass
        print(result)
        print(e_result)
        player_info = {
            "HP":v_HP,
            "MP":v_MP,
            "MY":v_MY
        }
        enermy_info = {
            "HP":e_HP,
            "MP":e_MP,
            "MY":e_MY
        }
            
        return player_info, enermy_info
    def click_p_btn(self, num):
        try:
            xpath = '//*[@id="main"]/div/div[3]/div[3]/div[7]/div'+'['+str(num)+']'
            btn = self.drive.find_element('xpath', xpath)
            btn.click()
        except:
            pass
    def play_card(self):
        play_card = self.drive.find_element("xpath", '//*[@id="main"]/div/div[3]/div[3]/div[8]/div')
        play_card.click()
    def get_cards(self):
        cards = []
        try:
            for num in range(1,20):
                target = self.drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[7]/div['+str(num)+']/div[2]')
                url = target.value_of_css_property(property_name='background-image')
                card = url.split("/")[6]
                card = card.replace('.png")', '') 
                card = card.replace("-",' ')
                if card == "fake":
                    target = self.drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[7]/div['+str(num)+']/div[1]')
                    url = target.value_of_css_property(property_name='background-image')
                    card = url.split("/")[6]
                    card = card.replace('.png")', '') 
                    card = card.replace("-",' ')
                    cards.append(card)
                cards.append(card)
        except:
            pass
        return cards
    def cards_info(self, card):
        for value in self.cards:
            if card == value[0]:
                target =  value
                target =  target[1:6]   
                target = np.array(target, dtype = int)
                return target
    def choose_card(self, target_card):
        target_num = 100
        try:
            for num in range(1,20):
                target = self.drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[7]/div['+str(num)+']/div[2]')
                url = target.value_of_css_property(property_name='background-image')
                card = url.split("/")[6]
                card = card.replace('.png")', '') 
                card = card.replace("-",' ')
                if card == target_card:
                    target_num = num
        except:
            print(f"Card {target_card} is not available")
            # target_num = 100   
        return target_num
    def print_cards_info():
        cards = webGame.get_cards()
        cards_info = webGame.cards_info(cards)
        cards_info_item = list(cards_info.keys())
        cards_info_value = list(cards_info.values())
        print("--------------------")
        for num in range(len(cards_info_value)):
            print("Card {} :".format(num+1))
            print(cards_info_item[num])
            print(cards_info_value[num])
            print("--------------------")
    def get_state(self):
        state_xpath = '//*[@id="main"]/div/div[3]/div[3]/div[3]/div[1]/div/span'
        try:
            state = self.drive.find_element('xpath', state_xpath)
            state_name = state.get_attribute('textContent')
            if state_name == self.player:
                self.state = 'active'
            if state_name != self.player:
                self.state = 'passive'
        except:
            sleep(1)
            self.state = 'changing'
        print('Game State: ', self.state)
    def eneymy_atk(self):
        try:
            enermy_atk = drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[3]/div[5]/span')
            enermy_atk = enermy_atk.get_attribute("textContent")
            enermy_atk = int(enermy_atk.replace('ATK', ''))
        except:
            enermy_atk = int(0)
        return enermy_atk
    def reset(self): #Restart the game
        global current_num
        current_num = 0
       
        try:
            start = self.drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[3]/div[2]')
            start.click()
            sleep(5)
            self.done = None
        except:
            pass
        obs = self.observation()
        return obs
    def observation(self):
        global current_card
        global current_player
        global current_enermy
        global current_num
        global obser
        flag = 0
        print('current number ', current_num)
        while 1 or flag ==10:
            sleep(0.5)
            cards_list = self.get_cards()
            _done = self.get_done()
            print(_done)
            if _done == True :
                obser = [0,0,8,0,0,0,0,0,0,0,0,0]
                print('001')
                return obser
            try:
                nbuy = self.drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[9]/div[2]')
                nbuy.click()
            except:
                pass 
            if len(cards_list)>0:
                flag+=1
                break
                
        self.get_state()
        while self.state == "changing":
            
            done = self.get_done()
            if done == True:
                obser = [0,0,8,0,0,0,0,0,0,0,0,0]
                return obser
            current_num = 0
            sleep(2)
            self.get_state()
        try:
            obser = self.cards_info(cards_list[current_num])
        except IndexError:
            done = self.get_done()
            if done == True:
                obser = [0,0,8,0,0,0,0,0,0,0,0,0]
                return obser
            current_num = 0
            obser = self.cards_info(cards_list[current_num])
        current_card = cards_list[current_num]
        player, enermy = self.get_condition()
        current_player = map(int, list(player.values()))
        current_enermy = map(int, list(enermy.values()))
        enermy_atk = self.eneymy_atk()
        result = list(obser)+list(current_player)+list(current_enermy)+list([enermy_atk])
        print('observation:', result)
        return np.array(result, dtype=np.int32)
    def get_done(self): #Get the imformations of game done
        try:
            done = self.drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[9]/span')
            if done.get_attribute("textContent") == "Finish Battle":
                print("Game over")
                sleep(1.5)
                self.done = True
                done_click = drive.find_element('xpath', '//*[@id="main"]/div/div[3]/div[3]/div[9]')
                done_click.click()
                sleep(2)
                return True
                
        except:
                self.done = False
                print("Game still going on...")           
        

In [8]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

NameError: name 'BaseCallback' is not defined

In [6]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [7]:
callback = TrainAndLoggingCallback(check_freq=100, save_path=CHECKPOINT_DIR)

NameError: name 'TrainAndLoggingCallback' is not defined

In [14]:
env = webGame(drive=drive)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\使用者\\桌面\\god field\\data\\ weapons.npy'

In [15]:
for epis in range(2):
    observation = env.reset()
    done = False
    total_reward = 0
    while  done == False:
        obs, reward, done, info = env.step(env.action_space.sample())
        total_reward += reward
    print(f'Reward of episode {epis}:{total_reward}')

NameError: name 'env' is not defined

In [ ]:
model = DQN("MlpPolicy", env,  tensorboard_log=LOG_DIR, verbose=1)
model.learn(total_timesteps= 10000, callback=callback)
# model.save("dqn_god_field")